## Preprocessing
Requirements:
- Modular for: 
  - Transform type
  - train/val/test directories
- General template: 
  - Isolate train/val/test datapoints
  - Transform each datapoint
  - Save to respective directories

Notes:
- ENSTDkAm and ENSTDkCl = non-synthesized recording directories